In [1]:
import numpy as np
import pandas as pd
import os
import time
import schedule
import joblib
import datetime
import multiprocessing as mp
from get_strategy_meta import get_all_strategy_meta
from query_api import get_path_dict, gen_connection, load_item_by_key, query_mysql
from get_signal_mongo import write_signal_mongo, get_signal_mongo
from get_signal_mysql import write_signal, get_signal
from mongo_functions import get_portfolio_db

Using TensorFlow backend.


In [2]:
strategy_meta_df = get_all_strategy_meta()

In [3]:
sdf = strategy_meta_df.loc[~strategy_meta_df["Backtest Status"].isin(["Abandon"])]
sdf["TF"] = sdf["TimeFrame"].apply(lambda x: int(x.split("_")[1]))
sdf = sdf.sort_values("TF")
# sdf


In [4]:
def do(db_variant, filter_tf):
    collection_name = "SignalOneE1"
    
    # Wait 5 seconds for data to arrive.
    time.sleep(5) 
    
    # Signal inference based on latest date
    query_date = datetime.datetime.now()

    # Initialize MongoDB (for writing signal)
    db = get_portfolio_db()
    
    # Filter strategies
    sdf2 = sdf.loc[sdf["TF"].isin(filter_tf)]
    ordered_strat_list = sdf2["Strategy"].values
    
    for strat in ordered_strat_list:
        print("Running Strategy: ", strat)
        strategy_meta = sdf2.loc[sdf2["Strategy"] == strat].to_dict(orient = "records")[0]
        PATH_DICT = get_path_dict(strategy_meta)
        keys = []
        for k1, v1 in PATH_DICT.items():
            for k2,v2 in v1.items():
                keys.append("{}_{}".format(k1, k2))
        for key in keys:
            p = mp.Process(target=write_signal, args=(query_date, db_variant, PATH_DICT, key, strategy_meta, db, collection_name))
            p.start()
            p.join()
        print("Completed Strategy: ", strat)
        
def do_mongo(filter_tf):
    output_db = "MLProduction"
    output_coll = "SignalOneF2"
    
    input_db = "Production"
    input_coll = "prices"
    
    # Wait 5 seconds for data to arrive.
    time.sleep(5) 
    
    # Signal inference based on latest date
    query_date = datetime.datetime.now()

    # Initialize MongoDB (for writing signal)
    db = get_portfolio_db(output_db)
    
    # Filter strategies
    sdf2 = sdf.loc[sdf["TF"].isin(filter_tf)]
    ordered_strat_list = sdf2["Strategy"].values
    
    for strat in ordered_strat_list:
        print("Running Strategy: ", strat)
        strategy_meta = sdf2.loc[sdf2["Strategy"] == strat].to_dict(orient = "records")[0]
        PATH_DICT = get_path_dict(strategy_meta)
        keys = []
        for k1, v1 in PATH_DICT.items():
            for k2,v2 in v1.items():
                keys.append("{}_{}".format(k1, k2))
        for key in keys:
            p = mp.Process(target=write_signal_mongo, args=(query_date, PATH_DICT, key, strategy_meta, db, output_coll, input_db, input_coll))
            p.start()
            p.join()
        print("Completed Strategy: ", strat)
        
def do_60_30_15_5(db_variant = "4", filter_tf = [5,15,30,60]):
#     do(db_variant, filter_tf)
    do_mongo(filter_tf)

def do_30_15_5(db_variant = "4", filter_tf = [5,15,30]):
    #     do(db_variant, filter_tf)
    do_mongo(filter_tf)
        
def do_15_5(db_variant = "4", filter_tf = [5,15]):
    #     do(db_variant, filter_tf)
    do_mongo(filter_tf)
        
def do_5(db_variant = "4", filter_tf = [5]):
    #     do(db_variant, filter_tf)
    do_mongo(filter_tf)

In [ ]:
# do(db_variant = "7", filter_tf = [15])



In [ ]:
do_mongo(filter_tf = [5,15,30,60])

In [ ]:

schedule.every(1).hour.at(':00').do(do_60_30_15_5)
schedule.every(1).hour.at(':05').do(do_5)
schedule.every(1).hour.at(':10').do(do_5)
schedule.every(1).hour.at(':15').do(do_15_5)
schedule.every(1).hour.at(':20').do(do_5)
schedule.every(1).hour.at(':25').do(do_5)
schedule.every(1).hour.at(':30').do(do_30_15_5)
schedule.every(1).hour.at(':35').do(do_5)
schedule.every(1).hour.at(':40').do(do_5)
schedule.every(1).hour.at(':45').do(do_15_5)
schedule.every(1).hour.at(':50').do(do_5)
schedule.every(1).hour.at(':55').do(do_5)

while 1:
    schedule.run_pending()
    time.sleep(1)